IDS 561 Group 24 Project Code

ML Model: XGBoost

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib

In [ ]:
datafile_for_modelling = pd.read_csv("datafile_for_modelling.csv")

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

In [ ]:
from sklearn.model_selection import train_test_split

#Using only 1 million rows
datafile_for_modelling = datafile_for_modelling.dropna()[:1000000]

#Getting label and feature columns
key_label = datafile_for_modelling['ARR_DELAY']
key_features = datafile_for_modelling[['DEP_DELAY', 'TAXI_OUT', 'CRS_ELAPSED_TIME', 'DISTANCE', 
                   'flight_weekday_1', 'flight_weekday_2', 'flight_weekday_3', 
                   'flight_weekday_4', 'flight_weekday_5', 'flight_weekday_6',
                   'flight_weekday_7']]

#Making test and train data sets
x_train, x_test, y_train, y_test = train_test_split(key_features, key_label, test_size = 0.20)

In [ ]:
pip install xgboost

     |████████████████████████████████| 1.7 MB 5.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer

model = xgb.XGBRegressor()

param = {'colsample_bytree': [1], 
         'gamma': [1.5], 
         'learning_rate': [0.04], 
         'max_depth': [8], 
         'min_child_weight': [3], 
         'n_estimators': [300],
         'silent': [1], 
         'subsample': [0.8]
         }

score = make_scorer(mean_squared_error, greater_is_better = False)

grid_search = GridSearchCV(estimator = model, param_grid = param, cv = 2,
                        n_jobs = 5,
                        verbose=True,
                        scoring = score)

#Fitting the grid search to the data
%time res = grid_search.fit(x_train, y_train)

best_parameters = res.best_params_

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[10:51:21] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


CPU times: user 7min 40s, sys: 48.1 s, total: 8min 28s
Wall time: 3min 6s


In [ ]:
print(res.best_score_)
print(res.best_params_)

-135.59675294462647
{'colsample_bytree': 1, 'gamma': 1.5, 'learning_rate': 0.04, 'max_depth': 8, 'min_child_weight': 3, 'n_estimators': 300, 'silent': 1, 'subsample': 0.8}


In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import math


%time preds = res.best_estimator_.predict(x_test)
print(r2_score(y_test, preds)) 
print(math.sqrt(mean_squared_error(y_test, preds))) 

CPU times: user 2.41 s, sys: 71.6 ms, total: 2.48 s
Wall time: 381 ms
0.9468582199149317
12.004458078371336
